In [ ]:
import openai
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

# Initialize LLM
llm = OpenAI(temperature=0.7)

# Load FAISS as a vector store for retrieval
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.load_local("faiss_index", embeddings=embedding_model)
retriever = vector_store.as_retriever()

# Step 1: High-Level Planner (Meta-Agent)
def high_level_planner(query):
    """Decomposes complex queries into sub-queries."""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an expert in query decomposition."},
                  {"role": "user", "content": f"Decompose this query into sub-queries: {query}"}]
    )
    return response['choices'][0]['message']['content'].split("\n")

# Step 2: Retrieval Agent
def retrieve_documents(sub_queries):
    """Retrieves documents for each sub-query."""
    retrieved_docs = {}
    for sub_query in sub_queries:
        docs = retriever.get_relevant_documents(sub_query)
        retrieved_docs[sub_query] = docs
    return retrieved_docs

# Step 3: Generator Agent
def generate_responses(retrieved_docs):
    """Generates responses for each retrieved document set."""
    responses = {}
    for sub_query, docs in retrieved_docs.items():
        context = "\n".join([doc.page_content for doc in docs])
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a domain expert."},
                {"role": "user", "content": f"Generate a response based on:\n{context}"}
            ]
        )
        responses[sub_query] = response['choices'][0]['message']['content']
    return responses

# Step 4: Final Synthesis Agent
def synthesize_final_response(sub_query_responses):
    """Merges multiple responses into a final coherent answer."""
    combined_text = "\n".join(sub_query_responses.values())
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert summarizer."},
            {"role": "user", "content": f"Create a final answer based on:\n{combined_text}"}
        ]
    )
    return response['choices'][0]['message']['content']

# Main Agentic RAG Function
def hierarchical_agentic_rag(query):
    """Executes the full hierarchical RAG pipeline."""
    sub_queries = high_level_planner(query)
    retrieved_docs = retrieve_documents(sub_queries)
    sub_query_responses = generate_responses(retrieved_docs)
    final_response = synthesize_final_response(sub_query_responses)
    return final_response

# Example Query
query = "Explain the impact of AI on healthcare and education"
final_output = hierarchical_agentic_rag(query)
print(final_output)
